<a href="https://colab.research.google.com/github/GeoTurkey/GMT_COURSES/blob/main/network_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network analysis in Python

Finding a shortest path using a specific street network is a common GIS problem that has many practical
applications. For example navigators are one of those "every-day" applications where **routing** using specific algorithms is used to find the optimal route between two (or multiple) points.

It is also possible to perform network analysis such as tranposrtation routing in Python.
[Networkx](https://networkx.github.io/documentation/stable/) is a Python module that provides tools for analyzing networks in various different ways. It also contains algorithms
such as [Dijkstra's algorithm](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.weighted.single_source_dijkstra.html#networkx.algorithms.shortest_paths.weighted.single_source_dijkstra) or
[A*](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.astar.astar_path.html#networkx.algorithms.shortest_paths.astar.astar_path) algoritm that are commonly used to find shortest paths along transportation network.

To be able to conduct network analysis, it is, of course, necessary to have a network that is used for the analyses. The [OSMnx](https://github.com/gboeing/osmnx) package enables us to retrieve routable networks from OpenStreetMap for various transport modes (walking, cycling and driving). OSMnx also combines some functionalities from `networkx` module to make it straightforward to conduct routing analysis based on OpenStreetMap data.

Next we will test the routing functionalities of OSMnx by finding a shortest path between two points based on drivable roads. With tiny modifications, it is also possible to repeat the analysis for the walkable street network.

## Get the network

Let's again start by importing the required modules

In [ ]:
! pip install contextily

In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from pyproj import CRS
import contextily as ctx
import warnings
warnings.filterwarnings('ignore')

When fetching network data from OpenStreetMap using OSMnx, it is possible to define the type of street network using the `network_type` parameter (see options from the [OSMnx documentation](https://osmnx.readthedocs.io/en/stable/osmnx.html?highlight=graph%20from%20place#osmnx.graph.graph_from_place)).
Let's download the OSM data from "Üniversiteler Mahallesi" and drivable roads.

In [ ]:
place_name = "Üniversiteler Mahallesi, Çankaya, Ankara"
# Retrieve the network
graph = ox.graph_from_place(place_name, network_type='drive')

In [ ]:
# plot the graph:
fig, ax = ox.plot_graph(graph)

Pro tip! Sometimes the shortest path might go slightly outside the defined area of interest. To account for this, we can fetch the network for a bit larger area than the district of Universiteler Mahallesi, in case the shortest path is not completely inside its boundaries.

In [ ]:
# Get the area of interest polygon
place_polygon = ox.geocode_to_gdf(place_name)

# Re-project the polygon to a local projected CRS
place_polygon = place_polygon.to_crs(epsg=32636)

# Buffer a bit
place_polygon["geometry"] = place_polygon.buffer(1000)

# Re-project the polygon back to WGS84, as required by osmnx
place_polygon = place_polygon.to_crs(epsg=4326)


In [ ]:
place_polygon

In [ ]:
# Retrieve the network
graph = ox.graph_from_polygon(place_polygon["geometry"].values[0], network_type='drive')

In [ ]:
place_polygon["geometry"].values[0]

Plot the graph:

In [ ]:
fig, ax = ox.plot_graph(graph)

Okey so now we have streets for the travel mode we specified earlier. Let's have a colser look at the attributes of the street network. Easiest way to do this is to convert the
graph (nodes and edges) into GeoDataFrames.

Converting graph into a GeoDataFrame can be done with function `graph_to_gdfs()` that we already used in previous tutorial. With parameters `nodes` and `edges`, it is possible to control whether to retrieve both nodes and edges from the graph.

In [ ]:
# Retrieve only edges from the graph
edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

In [ ]:
# Check columns
edges.columns

In [ ]:
# Check crs
edges.crs

Note that the CRS of the GeoDataFrame is be WGS84 (epsg: 4326).

In [ ]:
edges.head()

Okey, so we have quite many columns in our GeoDataFrame. Most of the columns are fairly self-explanatory but the following table describes all of them.
Most of the attributes come directly from the OpenStreetMap, however, columns `u` and `v` are Networkx specific ids. You can click on the links to get more information about each attribute:


| Column                                                     | Description                 | Data type         |
|------------------------------------------------------------|-----------------------------|-------------------|
| [bridge](http://wiki.openstreetmap.org/wiki/Key:bridge)    | Bridge feature              | boolean           |
| geometry                                                   | Geometry of the feature     | Shapely.geometry  |
| [highway](http://wiki.openstreetmap.org/wiki/Key:highway)  | Tag for roads (road type)   | str / list        |
| [lanes](http://wiki.openstreetmap.org/wiki/Key:lanes)      | Number of lanes             | int (or nan)      |
| [lenght](http://wiki.openstreetmap.org/wiki/Key:length)    | Length of feature (meters)  | float             |
| [maxspeed](http://wiki.openstreetmap.org/wiki/Key:maxspeed)| maximum legal speed limit   | int /list         |
| [name](http://wiki.openstreetmap.org/wiki/Key:name)        | Name of the (street) element| str (or nan)      |
| [oneway](http://wiki.openstreetmap.org/wiki/Key:oneway)    | One way road                | boolean           |
| [osmid](http://wiki.openstreetmap.org/wiki/Node)           | Unique ids for the element  | list              |
| [u](http://ow.ly/bV8n30h7Ufm)                              | The first node of edge      | int               |
| [v](http://ow.ly/bV8n30h7Ufm)                              | The last node of edge       | int               |


Let's take a look what kind of features we have in the `highway` column:

In [ ]:
edges['highway'].value_counts()

I now we can confirm that as a result our street network indeed only contains such streets where it is allowed to drive with a car as there are no e.g. cycleways or footways included in the data.

As the data is in WGS84 format, we might want to reproject our data into a metric system before proceeding to the shortest path analysis.
We can re-project the graph from latitudes and longitudes to an appropriate UTM zone using the [project_graph()](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.projection.project_graph) function from OSMnx.

In [ ]:
# Project the data
graph_proj = ox.project_graph(graph)

In [ ]:
# Get Edges and Nodes
nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)

In [ ]:
print("Coordinate system:", edges_proj.crs)
print("Coordinate system:", nodes_proj.crs)

In [ ]:
nodes_proj.head()

Okey, as we can see from the CRS the data is now in [UTM projection](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) using zone 36 which is the one used for TURKEY, and indeed the orientation of the map and the geometry values also confirm this.


Furthermore, we can check the epsg code of this projection using pyproj CRS:

In [ ]:
CRS(edges_proj.crs).to_epsg()

Indeed, the projection is now [WGS 84 / UTM zone 36N, EPSG:32636](https://epsg.io/32636).

## Shortest path analysis

Let's now calculate the shortest path between two points using the [shortest path function in Networkx](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.generic.shortest_path.html#shortest-path).

#### Origin and destination points

First we need to specify the source and target locations for our route. If you are familiar with the Universiteler MAhallesi area, you can specify a custom placename as a source location. Or, you can follow along and choose these points as the origin and destination in the analysis:
- [Beytepe Gün Hastanesi](https://nominatim.openstreetmap.org/ui/search.html?q=Beytepe+G%C3%BCn) - hospital area and current startup hub.
- [Yükseköğretim Kurulu](https://nominatim.openstreetmap.org/ui/search.html?q=Y%C3%BCksek%C3%B6%C4%9Fretim+Kurulu) - YÖK building in Bilkent.

We could figure out the coordinates for these locations manually, and create shapely points based on the coordinates.
However,  it is more handy to fetch the location of our source destination directly from OSM:

In [ ]:
# Set place name
placename = "Beytepe Gün Hastanesi"

In [ ]:
# Geocode the place name
geocoded_place = ox.geocode_to_gdf(placename)

In [ ]:
# Check the result
geocoded_place

In [ ]:
geocoded_place.crs

As output, we received the building footprint. From here, we can get the centroid as the source location of our shortest path analysis. However, we first need to project the data into the correct crs:

In [ ]:
# Re-project into the same CRS as the road network
geocoded_place = geocoded_place.to_crs(CRS(edges_proj.crs))

In [ ]:
geocoded_place.crs

In [ ]:
geocoded_place

In [ ]:
# Get centroid as shapely point
origin = geocoded_place["geometry"].centroid.values[0]

In [ ]:
print(origin)

Great! Now we have defined the origin point for our network analysis.
We can repeat the same steps to retrieve central point of *ruttopuisto*-park as the destination:

In [ ]:
# Set place name
placename = "Yükseköğretim Kurulu"

# Geocode the place name
geocoded_place = ox.geocode_to_gdf(placename)

# Re-project into the same CRS as the road network
geocoded_place = geocoded_place.to_crs(CRS(edges_proj.crs))

# Get centroid of the polygon as shapely point
destination = geocoded_place["geometry"].centroid.values[0]

print(destination)

Now we have shapely points representing the origin and destination locations for our network analysis. Next step is to find these points on the routable network before the final routing.

#### Nearest node

Let's now find the nearest graph nodes (and their node IDs) to these points using OSMnx [distance.nearest_nodes](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_nodes).
As a starting point, we have the two Shapely Point objects we just defined as the origin and destination locations.

Find the nearest node to a point or to each of several points.

If X and Y are single coordinate values, this will return the nearest node to that point. If X and Y are lists of coordinate values, this will return the nearest node to each point.

If the graph is projected, this uses a k-d tree for euclidean nearest neighbor search, which requires that scipy is installed as an optional dependency. If it is unprojected, this uses a ball tree for haversine nearest neighbor search, which requires that scikit-learn is installed as an optional dependency.

In [ ]:
# Find the node in the graph that is closest to the origin point (here, we want to get the node id)
orig_node_id = ox.distance.nearest_nodes(graph_proj, origin.x, origin.y)
orig_node_id

In [ ]:
# Find the node in the graph that is closest to the target point (here, we want to get the node id)
target_node_id = ox.distance.nearest_nodes(graph_proj, destination.x, destination.y)
target_node_id

Now we have the IDs for the closest nodes that were found from the graph to the origin and target points that we specified.

Let's retrieve the node information from the `nodes_proj` GeoDataFrame by passing the ids to the `loc` indexer

In [ ]:
nodes_proj.head()

In [ ]:
# Retrieve the rows from the nodes GeoDataFrame based on the node id (node id is the index label)
orig_node = nodes_proj.loc[orig_node_id]
target_node = nodes_proj.loc[target_node_id]

In [ ]:
orig_node

In [ ]:
target_node

Let's also create a GeoDataFrame that contains these points

In [ ]:
# Create a GeoDataFrame from the origin and target points
od_nodes = gpd.GeoDataFrame([orig_node, target_node], geometry='geometry', crs=nodes_proj.crs)
od_nodes.head()

Okay, as a result we got now the closest node IDs of our origin and target locations. As you can see, the `index` in this GeoDataFrame corresponds to the IDs that we found with `distance.nearest_nodes()` function.

#### Routing

Now we are ready to do the routing and find the shortest path between the origin and target locations
by using the `shortest_path()` [function](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.shortest_paths.generic.shortest_path.html) of networkx.
With `weight` -parameter we can specify that `'length'` attribute should be used as the cost impedance in the routing. If specifying the weight parameter, NetworkX will use by default Dijkstra's algorithm to find the optimal route. We need to specify the graph that is used for routing, and the origin `ID` (*source*) and the target `ID` in between the shortest path will be calculated:


In [ ]:
# Calculate the shortest path
route = nx.shortest_path(G=graph_proj, source=orig_node_id, target=target_node_id, weight='length')

# Show what we have
print(route)

As a result we get a list of all the nodes that are along the shortest path.

- We could extract the locations of those nodes from the `nodes_proj` GeoDataFrame and create a LineString presentation of the points, but luckily, OSMnx can do that for us and we can plot shortest path by using `plot_graph_route()` function:


In [ ]:
# Plot the shortest path
fig, ax = ox.plot_graph_route(graph_proj, route)

Nice! Now we have the shortest path between our origin and target locations.
Being able to analyze shortest paths between locations can be valuable information for many applications.
Here, we only analyzed the shortest paths based on distance but quite often it is more useful to find the
optimal routes between locations based on the travelled time. Here, for example we could calculate the time that it takes to cross each road segment by dividing the length of the road segment with the speed limit and calculate the optimal routes by taking into account the speed limits as well that might alter the result especially on longer trips than here.

## Saving shortest paths to disk

Quite often you need to save the route into a file for further analysis and visualization purposes, or at least have it as a GeoDataFrame object in Python.
Hence, let's continue still a bit and see how we can turn the route into a linestring and save the shortest path geometry and related attributes into a geopackage file.

- First we need to get the nodes that belong to the shortest path:


In [ ]:
# Get the nodes along the shortest path
route_nodes = nodes_proj.loc[route]
route_nodes

As we can see, now we have all the nodes that were part of the shortest path as a GeoDataFrame.

- Now we can create a LineString out of the Point geometries of the nodes:

In [ ]:
from shapely.geometry import LineString, Point

# Create a geometry for the shortest path
route_line = LineString(list(route_nodes["geometry"].values))
route_line

In [ ]:
print(route_line)

Now we have the route as a LineString geometry.

- Let's make a GeoDataFrame out of it having some useful information about our route such as a list of the osmids that are part of the route and the length of the route.

In [ ]:
# Create a GeoDataFrame
route_geom = gpd.GeoDataFrame([[route_line]], geometry='geometry', crs=edges_proj.crs, columns=['geometry'])
route_geom

In [ ]:
# Add a list of osmids associated with the route
route_geom.loc[0, 'osmids'] = str(list(route_nodes.index.values))
route_geom

In [ ]:

# Calculate the route length
route_geom['length_m'] = route_geom.length
route_geom

Now we have a GeoDataFrame that we can save to disk. Let's still confirm that everything is ok by plotting our route on top of our street network and some buildings, and plot also the origin and target points on top of our map.

- Get buildings:

In [ ]:
tags = {'building': True}
buildings = ox.geometries_from_place(place_name, tags)

re-project buildings

In [ ]:
buildings_proj = buildings.to_crs(CRS(edges_proj.crs))

- Let's now plot the route and the street network elements to verify that everything is as it should:

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

# Plot edges and nodes
edges_proj.plot(ax=ax, linewidth=0.75, color='gray')
nodes_proj.plot(ax=ax, markersize=2, color='gray')

# Add buildings
ax = buildings_proj.plot(ax=ax, facecolor='gray', alpha=0.7)

# Add the route
ax = route_geom.plot(ax=ax, linewidth=2, linestyle='--', color='red')

# Add the origin and destination nodes of the route
ax = od_nodes.plot(ax=ax, markersize=30, color='red')

# Add basemap
ctx.add_basemap(ax, crs=buildings_proj.crs, source=ctx.providers.CartoDB.Positron)

Great everything seems to be in order! As you can see, now we have a full control of all the elements of our map and we can use all the aesthetic properties that matplotlib provides to modify how our map will look like. Now we are almost ready to save our data into disk.


## Prepare data for saving to file

The data contain certain data types (such as `list` or `boolean`) that should be converted into character strings prior to saving the data to file.Another option would be to drop invalid columns.

In [ ]:
edges_proj.head()

In [ ]:
# Columns with invalid values
problematic_columns = [
    "osmid",
    "lanes",
    "name",
    "highway",
    "maxspeed",
    "reversed",
    "junction",
    "bridge",
    "tunnel",
    "access",
    "service",

]

#  convert selected columns to string format
edges[problematic_columns] = edges[problematic_columns].astype(str)


In [ ]:
route_geom

In [ ]:
route_geom["osmids"] = route_geom["osmids"].astype(str)


## Save the data:

In [ ]:
# Save one layer after another
output_gpkg = "Universiteler_Mah.gpkg"

edges.to_file(output_gpkg, layer="streets")
route_geom.to_file(output_gpkg, layer="route")
nodes_proj.to_file(output_gpkg, layer="nodes")
buildings[['geometry', 'name', 'addr:street']].to_file(output_gpkg, layer="buildings")


Great, now we have saved all the data that was used to produce the maps into a geopackage.